In [ ]:
import cv2
import numpy as np
import os
from scipy.spatial import distance

# 1. 식판 좌표 미리 정의
# 각 칸의 좌상단 (x1, y1)과 우하단 (x2, y2) 좌표
plate_coords = {
    "반찬1": (50, 50, 150, 150),
    "반찬2": (200, 50, 300, 150),
    "반찬3": (50, 200, 150, 300),
    "반찬4": (200, 200, 300, 300),
    "국": (350, 100, 450, 200),
    "밥": (350, 250, 450, 350)
}

# 2. Segment된 영역 중심 계산 함수
def calculate_centroid(segment_mask):
    """
    분할된 마스크에서 중심 좌표 계산
    """
    M = cv2.moments(segment_mask)
    if M["m00"] == 0:  # 분할 영역이 없는 경우
        return None
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])
    return (cx, cy)

# 3. 가장 가까운 좌표 선택 함수
def find_closest_section(centroid, plate_coords):
    """
    Segment 중심 좌표와 가장 가까운 식판 좌표를 찾음
    """
    min_distance = float("inf")
    closest_section = None

    for section, (x1, y1, x2, y2) in plate_coords.items():
        # 식판 칸의 중심 계산
        section_center = ((x1 + x2) // 2, (y1 + y2) // 2)
        dist = distance.euclidean(centroid, section_center)
        if dist < min_distance:
            min_distance = dist
            closest_section = section

    return closest_section

# 4. 이미지 Crop 및 저장 함수
def crop_and_save(image, section, plate_coords, save_dir):
    """
    이미지를 해당 칸으로 Crop하여 저장
    """
    x1, y1, x2, y2 = plate_coords[section]
    cropped_image = image[y1:y2, x1:x2]
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{section}.jpg")
    cv2.imwrite(save_path, cropped_image)
    print(f"{section} 영역이 {save_path}에 저장되었습니다.")

# 5. Main 코드
def process_image(image_path, segment_mask_path, save_dir):
    """
    입력 이미지에서 분할된 영역을 기반으로 가장 가까운 식판 좌표로 Crop하여 저장
    """
    # 이미지와 분할 마스크 읽기
    image = cv2.imread(image_path)
    segment_mask = cv2.imread(segment_mask_path, cv2.IMREAD_GRAYSCALE)

    # Segment된 영역 중심 계산
    centroid = calculate_centroid(segment_mask)
    if centroid is None:
        print("Segment된 영역이 없습니다.")
        return

    # 가장 가까운 식판 칸 찾기
    closest_section = find_closest_section(centroid, plate_coords)

    # Crop 및 저장
    crop_and_save(image, closest_section, plate_coords, save_dir)

# 6. 테스트 실행
image_path = "input_image.jpg"  # 원본 이미지 경로
segment_mask_path = "segment_mask.png"  # 분할 마스크 경로
save_dir = "cropped_images"  # 저장할 디렉토리

process_image(image_path, segment_mask_path, save_dir)


### 클래스별 양 코드 하위폴더 생성 후 crop image 저장하기

In [ ]:
import os
import pandas as pd
import cv2

def save_image_with_folder_structure(csv_path, image, image_id, save_dir):
    """
    CSV 파일을 참고하여 이미지를 지정된 폴더 구조에 저장.
    
    Parameters:
        csv_path (str): 실시간 음식정보 CSV 파일 경로.
        image (numpy.ndarray): 저장할 이미지 데이터.
        image_id (str): 이미지 고유번호.
        save_dir (str): 기본 저장 디렉토리.
    """
    # CSV 파일 읽기
    try:
        csv_data = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"CSV 파일 {csv_path}을(를) 찾을 수 없습니다.")
        return

    # 0번째 컬럼에서 상위 폴더(음식명 ID)와 하위 폴더(양 추정 코드) 추출
    for idx, row in csv_data.iterrows():
        try:
            column_value = row[0]  # 0번째 컬럼 값
            unique_id = str(row[1])  # 고유번호

            if '_' in column_value:
                food_name_id, volume_code = column_value.split('_')
            else:
                print(f"'{column_value}'는 '_' 기준으로 분리할 수 없습니다.")
                continue

            # 폴더 경로 생성
            target_folder = os.path.join(save_dir, food_name_id, volume_code)
            os.makedirs(target_folder, exist_ok=True)

            # 이미지 저장 경로
            save_path = os.path.join(target_folder, f"{unique_id}.jpg")

            # 이미지 저장
            cv2.imwrite(save_path, image)
            print(f"이미지 {save_path}에 저장되었습니다.")

        except Exception as e:
            print(f"오류 발생: {e}")
            continue

# 예제 실행
csv_path = "real_time_food_info.csv"  # 실시간 음식정보 CSV 파일 경로
image_path = "input_image.jpg"  # 저장할 이미지 경로
save_dir = "food_images"  # 저장할 최상위 폴더

# 이미지 로드 (예제 이미지)
image = cv2.imread(image_path)

# 고유 이미지 ID (1번째 컬럼 값)
image_id = "12345"  # 예제 ID

# 이미지 저장 처리
save_image_with_folder_structure(csv_path, image, image_id, save_dir)
